In [0]:
!pip3 install ivis

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from ivis import Ivis
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

from keras.layers import BatchNormalization, Dense, Activation, Lambda, Input
from keras.models import Sequential, Model
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from imblearn.over_sampling import SMOTE, RandomOverSampler

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB

import itertools
import time

base_path = './drive/My Drive/'

train_data_fname = 'xerces-1.2.csv'
test_data_fname = 'xerces-1.3.csv'

batch_size = 128
num_classes = 2

train_df = pd.read_csv(f'{base_path}/{train_data_fname}')
test_df = pd.read_csv(f'{base_path}/{test_data_fname}')

train_df.head()

x_train = train_df.drop('bug', axis=1)
x_train.drop(['name', 'version', 'name.1'], axis=1, inplace=True)
y_train = train_df['bug']

x_test = test_df.drop('bug', axis=1)
x_test.drop(['name', 'version', 'name.1'], axis=1, inplace=True)
y_test = test_df['bug']

(y_train == 0).sum(), (y_train == 1).sum(), (y_train == 2).sum()

idx = np.where(y_train == 2)[0]
x_train.drop(idx, axis=0, inplace=True)
y_train.drop(idx, inplace=True)

x_train.shape, y_train.shape

idx = np.where(y_test > 1)[0]
x_test.drop(idx, axis=0, inplace=True)
y_test.drop(idx, inplace=True)

x_test.shape, y_test.shape

np.unique(y_train)

[(i, (y_test == i).sum()) for i in np.unique(y_test)]



"""## Helper functions"""

def parabola(x):
    return x ** 2

def get_model(units=2, n_layers=1, deep_learner=False, batchnorm=False, activation='relu'):
    """
    Returns a model.
    
    Params:
    =======
    units: int - Number of input units. Only for deep learners.
    layers: int - Number of hidden layers. Only for deep learners.
    deep_learner: boolean - To use deep learner, set to True.
    batchnorm: boolean - If True, uses Batch Normalization.
    
    Returns:
    ========
    An instance of keras.Sequential or AdaBoost from sklearn.
    """
    if deep_learner:
        layers = []
        
        # Add first hidden layer set.
        first = Dense(units, input_shape=(units,))
        layers.append(first)
        
        if batchnorm:
            layers.append(BatchNormalization())
        
        layers.append(Activation(activation))
        
        # Add the other hidden layers.
        for i in range(n_layers - 1):
            layers.append(Dense(units))
            
            if batchnorm:
                layers.append(BatchNormalization())
            
            layers.append(Activation(activation))
        
        # Add the output layer.
        layers.append(Dense(1))
        layers.append(Activation('sigmoid'))
        
        return Sequential(layers)
    else:
        return AdaBoostClassifier()

def get_oversampler(name=None):
    """
    Returns an oversampler.
    
    Params:
    =======
    name: str or None - One of None, 'smote', and 'random'.
    """
    if name is None:
        return None
    if name == 'smote':
        return SMOTE()
    if name == 'random':
        return RandomOverSampler()

def run_one_experiment(x_resampled, y_resampled, x_test, y_test, deep_learner=False, n_layers=1, batchnorm=False, lipschitz_lr=False, 
                       reduce_data=False, embedding_dims=2, k=7, oversample=False, oversampler=None,
                       activation='relu'):
    """
    Runs one experiment.
    
    Params:
    =======
    x, y: np.array - Input data
    deep_learner: boolean - If True, uses a deep learner.
    layers: int - Number of hidden layers. Deep learners only.
    batchnorm: boolean - If True, uses Batch Normalization in deep learner.
    lipschitz_lr: boolean - If True, uses LipschitzLR
    reduce_data: boolean - If True, reduces data using Ivis.
    k: int - k used in Ivis reduction.
    embedding_dims: int - Number of dimensions to reduce data to
    oversample: boolean - If True, oversamples dataset.
    oversampler: str or None - One of None, 'smote', and 'random'.
    
    Returns:
    ========
    prec, recall, f1, time - Precision, recall, and F-1 scores of the model, along with runtime
    """    
    
    # Step 3: Get the model
    model = get_model(units=x_resampled.shape[1],n_layers=n_layers, 
                      deep_learner=deep_learner, batchnorm=batchnorm,
                      activation=activation)
    
    # Step 4: Train model
    if deep_learner:
        ####################
        # LipschitzLR code #
        ####################    
        def lr_schedule(epoch):
            """Learning Rate Schedule
            # Arguments
                epoch (int): The number of epochs
            # Returns
                lr (float32): learning rate
            """

            Kz = 0.
            for i in range((len(x_resampled) - 1) // batch_size + 1):
                start_i = i * batch_size
                end_i = start_i + batch_size
                xb = x_resampled[start_i:end_i]

                activ = np.linalg.norm(func([xb]))
                if activ > Kz:
                    Kz = activ

            K_ = ((num_classes - 1) * Kz) / (num_classes * batch_size)
            lr = 1 / K_
            #print('Epoch', epoch + 1, 'LR =', lr)
            return lr
        ####################
        
        if lipschitz_lr:
            model.compile(loss='binary_crossentropy',
                  optimizer='sgd')
            func = K.function([model.layers[0].input], [model.layers[-2].output])
            lr_scheduler = LearningRateScheduler(lr_schedule)
            
            start = time.time()
            model.fit(x_resampled, y_resampled, batch_size=batch_size, verbose=0, 
                      validation_data=(x_test, y_test), epochs=100, callbacks=[lr_scheduler])
            end = time.time()
        else:
            model.compile(loss='binary_crossentropy',
                  optimizer='adam')
            
            start = time.time()
            model.fit(x_resampled, y_resampled, batch_size=batch_size, verbose=0, 
                      validation_data=(x_test, y_test), epochs=100)
            end = time.time()
        
    else:
        start = time.time()
        model.fit(x_resampled, y_resampled)
        end = time.time()
    
    # Step 5: Evaluate model
    evaluators = [precision_score, recall_score, f1_score]
    if deep_learner:
        prec, recall, f1 = [f(y_test, model.predict_classes(x_test)) for f in evaluators]
    else:
        prec, recall, f1 = [f(y_test, model.predict(x_test)) for f in evaluators]
    
    return prec, recall, f1, (end - start)

def run_20_experiments(reduce_data=False, oversample=False, oversampler=None, 
                       embedding_dims=2, k=7, lipschitz_lr=False, activation='relu', **kwargs):
    global x_train, y_train, x_test, y_test
    prec = []
    recall = []
    f1 = []
    times = []
    
    count = 0

    # Step 1: Reduce dimensions --> (x_*_reduced, y_*)
    x_train_reduced = np.array(x_train)
    x_test_reduced = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    if reduce_data:
        ivis = Ivis(embedding_dims=embedding_dims, k=k, verbose=0)
        x_train_reduced = ivis.fit_transform(x_train_reduced)
        x_test_reduced = ivis.transform(x_test_reduced)

    # Step 2: Oversample data --> (x_resampled, y_resampled)
    x_resampled, y_resampled = x_train_reduced, y_train
    if oversample:
        sampler = get_oversampler(name='random')
        x_resampled, y_resampled = sampler.fit_resample(x_train_reduced, y_train)

    while count != 20:
        try:          
            precision, rec, f1_, time_ = run_one_experiment(x_resampled, y_resampled,
                                                            x_test_reduced, y_test, 
                                                            lipschitz_lr=lipschitz_lr, 
                                                            activation=activation, **kwargs)
            prec.append(precision)
            recall.append(rec)
            f1.append(f1_)
            times.append(time_)
            count += 1
        except KeyboardInterrupt:
            raise
        except:
            print('Retrying...')
            if lipschitz_lr:
                pass
            else:
                raise
    
    print('Precision:', prec, '\nRecall:', recall, '\nF1:', f1, '\nRuntime:', times)

"""## Run recommended experiments"""

def get_experimental_config():
    configs = [
        {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'activation': parabola},
        {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': parabola},
        {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'},
        {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'activation': parabola},
        {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
    ]

    for config in configs:
        # Skip over completed configs
        complete = [
        ]

        if config not in complete:
            yield config

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
count = 0
for _ in get_experimental_config():
    count += 1
print(count * 20)

580


In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}













Precision: [0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.1497584541062802, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715, 0.13839285714285715] 
Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.24313725490196078, 0.24313725490196078, 0.24313725490196078, 0.24313725490196078, 0.24313725490196078, 0.24313725490196078, 0.24313725490196078, 0.24313725490196078, 0.24313725490196078, 0.2605042016806723, 0.24313725490196078, 0.24313725490196078, 0.24313725490196078,

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.1514629948364888, 0.0, 0.208, 0.0, 0.13043478260869565, 0.1838074398249453, 0.1975, 0.0, 0.1111111111111111, 0.17938144329896907, 0.0, 0.0, 0.05263157894736842, 0.25, 0.0, 0.0, 0.0, 0.1566265060240964, 0.0, 0.1111111111111111] 
Recall: [0.946236559139785, 0.0, 0.8387096774193549, 0.0, 0.03225806451612903, 0.9032258064516129, 0.8494623655913979, 0.0, 0.010752688172043012, 0.9354838709677419, 0.0, 0.0, 0.010752688172043012, 0.021505376344086023, 0.0, 0.0, 0.0, 0.978494623655914, 0.0, 0.010752688172043012] 
F1: [0.26112759643916916, 0.0, 0.3333333333333333, 0.0, 0.05172413793103448, 0.3054545454545455, 0.3204868154158216, 0.0, 0.0196078431372549, 0.30103806228373703, 0.0, 0.0, 0.017857142857142856, 0.03960396039603961, 0.0, 0.0, 0.0, 0.27002967359050445, 0.0, 0.0196078431372549] 
Runtime: [60.28896188735962, 60.84834289550781, 60.34929084777832, 60.47072720527649, 60.52225685119629, 60.55118441581726, 60.695770502090454, 62.87689781188965, 61.29481744766235, 61.6004695892334

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.0, 0.0, 0.0, 0.0, 0.0, 0.2857142857142857, 0.0, 0.18461538461538463, 0.0, 0.058823529411764705, 0.0, 0.0, 0.358974358974359, 0.1708253358925144, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0] 
Recall: [0.0, 0.0, 0.0, 0.0, 0.0, 0.06451612903225806, 0.0, 0.6451612903225806, 0.0, 0.010752688172043012, 0.0, 0.0, 0.3010752688172043, 0.956989247311828, 0.0, 0.0, 0.010752688172043012, 0.0, 0.0, 0.0] 
F1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.10526315789473685, 0.0, 0.28708133971291866, 0.0, 0.01818181818181818, 0.0, 0.0, 0.3274853801169591, 0.2899022801302931, 0.0, 0.0, 0.020408163265306128, 0.0, 0.0, 0.0] 
Runtime: [57.96222901344299, 57.969430923461914, 58.12081813812256, 58.1535701751709, 58.41496825218201, 58.206053733825684, 58.68410921096802, 58.589566707611084, 58.829129219055176, 58.76607942581177, 58.16111445426941, 57.99653744697571, 57.965558767318726, 58.22095799446106, 58.28080439567566, 58.35223603248596, 58.39401626586914, 58.76834416389465, 58.8053514957428, 58.58117365837097]



In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








Precision: [0.2529182879377432, 0.2625482625482625, 0.2802197802197802, 0.23297491039426524, 0.24380165289256198, 0.2636363636363636, 0.2425531914893617, 0.2545454545454545, 0.2084507042253521, 0.273542600896861, 0.23381294964028776, 0.2669491525423729, 0.26956521739130435, 0.23859649122807017, 0.24555160142348753, 0.25877192982456143, 0.24822695035460993, 0.2532188841201717, 0.2594594594594595, 0.26046511627906976] 
Recall: [0.6989247311827957, 0.7311827956989247, 0.5483870967741935, 0.6989247311827957, 0.6344086021505376, 0.6236559139784946, 0.6129032258064516, 0.6021505376344086, 0.7956989247311828, 0.6559139784946236, 0.6989247311827957, 0.6774193548387096, 0.6666666666666666, 0.7311827956989247, 0.74193548387

## Camel 1.2-1.4 experiments

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Precision: [0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423] 
Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.16340621403912542, 0.16340621403912542, 0.16340621403912542, 0.16340621403912542, 0.16340621403912542, 0.16340621403912542, 0.16340621403912542, 0.16340621403912542, 0.16340621403912542, 0.16340621403

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 15, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 15 : {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'activation': <function parabola at 0x7f0fe9993510>}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.08964646464646464, 0.08935611038107753, 0.08897243107769423, 0.08838383838383838, 0.08897243107769423, 0.08816120906801007, 0.08897243107769423, 0.09137709137709138, 0.08897243107769423, 0.08897243107769423, 0.11585365853658537, 0.08897243107769423, 0.08897243107769423, 0.08897243107769423, 0.0890840652446675, 0.08903225806451613, 0.09259259259259259, 0.08897243107769423, 0.08897243107769423, 0.08306709265175719] 
Recall: [1.0, 0.9577464788732394, 1.0, 0.9859154929577465, 1.0, 0.9859154929577465, 1.0, 1.0, 1.0, 1.0, 0.5352112676056338, 1.0, 1.0, 1.0, 1.0, 0.971830985915493, 0.98591549295774

## Camel 1.4-1.6 Experiments

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.112565445026178, 0.1121251629726206, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027, 0.11503416856492027] 
Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8514851485148515, 0.8514851485148515, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.20633299284984677, 0.20633299284984677, 0.20633299284984677, 0.206

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.0, 0.1891891891891892, 0.2857142857142857, 0.08823529411764706, 0.12264150943396226, 0.3333333333333333, 0.5, 0.0, 0.6666666666666666, 0.0967741935483871, 0.10256410256410256, 0.10112359550561797, 0.4, 0.1165644171779141, 0.2857142857142857, 0.15384615384615385, 0.08181818181818182, 0.25, 0.0, 0.0] 
Recall: [0.0, 0.06930693069306931, 0.019801980198019802, 0.0297029702970297, 0.12871287128712872, 0.009900990099009901, 0.009900990099009901, 0.0, 0.019801980198019802, 0.0594059405940594, 0.1188118811881188, 0.0891089108910891, 0.019801980198019802, 0.18811881188118812, 0.019801980198019802, 0.039603960396039604, 0.0891089108910891, 0.009900990099009901, 0.0, 0.0] 
F1: [0.0, 0.10144927536231885, 0.037037037037037035, 0.04444444444444444, 0.12560386473429952, 0.019230769230769232, 0.019417475728155338, 0.0, 0.038461538461538464, 0.07361963190184048, 0.11009174311926605, 0.09473684210526315, 0.03773584905660377, 0.14393939393939395, 0.037037037037037035, 0.06299212598425197, 0.

## Ivy 1.4-2.0

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.03488372093023256, 0.08235294117647059, 0.07421875, 0.03349282296650718, 0.07738095238095238, 0.02158273381294964, 0.18181818181818182, 0.08235294117647059, 0.10426540284360189, 0.08235294117647059, 0.08235294117647059, 0.01818181818181818, 0.08235294117647059, 0.08235294117647059, 0.07053941908713693, 0.07643312101910828, 0.08636363636363636, 0.08235294117647059, 0.08235294117647059, 0.07911392405063292] 
Recall: [0.21428571428571427, 1.0, 0.6785714285714286, 0.25, 0.9285714285714286, 0.10714285714285714, 0.7142857142857143, 1.0, 0.7857142857142857, 1.0, 1.0, 0.10714285714285714, 1.0, 1.0, 0.6071428571428571, 0.857142857

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.10714285714285714, 0.0, 0.03614457831325301, 0.15492957746478872, 0.0, 0.0, 0.18018018018018017, 0.21176470588235294, 0.1073170731707317, 0.09090909090909091, 0.0, 0.11483253588516747, 0.0641025641025641, 0.0, 0.09398496240601503, 0.12626262626262627, 0.07865168539325842, 0.06976744186046512, 0.03608247422680412, 0.0] 
Recall: [0.9642857142857143, 0.0, 0.32142857142857145, 0.7857142857142857, 0.0, 0.0, 0.7142857142857143, 0.6428571428571429, 0.7857142857142857, 0.8928571428571429, 0.0, 0.8571428571428571, 0.17857142857142858, 0.0, 0.8928571428571429, 0.8928571428571429, 0.25, 0.42857142857142855, 0.25, 0.0] 
F1: [0.19285714285714284, 0.0, 0.06498194945848375, 0.2588235294117647, 0.0, 0.0, 0.28776978417266186, 0.3185840707964602, 0.18884120171673818, 0.16501650165016504, 0.0, 0.20253164556962025, 0.09433962264150943, 0.0, 0.17006802721088435, 0.22123893805309736, 0.11965811965811966, 0.12, 0.06306306306306306, 0.0] 
Runtime: [34.482478857040405, 35.35021686553955, 34.51913

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.3333333333333333, 0.0, 0.034482758620689655, 0.0, 0.125, 0.0, 0.125, 0.0, 0.18181818181818182, 0.0, 0.16666666666666666, 0.0, 0.0, 0.11428571428571428, 0.0, 0.05263157894736842, 0.0, 0.11904761904761904, 0.16666666666666666, 0.0] 
Recall: [0.03571428571428571, 0.0, 0.03571428571428571, 0.0, 0.03571428571428571, 0.0, 0.03571428571428571, 0.0, 0.07142857142857142, 0.0, 0.03571428571428571, 0.0, 0.0, 0.14285714285714285, 0.0, 0.03571428571428571, 0.0, 0.17857142857142858, 0.03571428571428571, 0.0] 
F1: [0.06451612903225806, 0.0, 0.03508771929824562, 0.0, 0.05555555555555556, 0.0, 0.05555555555555556, 0.0, 0.10256410256410255, 0.0, 0.058823529411764705, 0.0, 0.0, 0.12698412698412698, 0.0, 0.0425531914893617, 0.0, 0.14285714285714285, 0.058823529411764705, 0.0] 
Runtime: [46.100263595581055, 45.92298603057861, 45.58269262313843, 46.75237703323364, 45.796157121658325, 46.15079927444458, 45.59069299697876, 45.6432945728302, 45.63221287727356, 47.94633364677429, 47.95758724212646

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
Recall: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03571428571428571, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07142857142857142, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
F1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.058823529411764705, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12121212121212122, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
Runtime: [53.17892813682556, 52.87765169143677, 54.013909578323364, 53.331153869628906, 53.613460540771484, 53.3358473777771, 53.999481439590454, 54.92594265937805, 53.88486313819885, 53.56878995895386, 54.33151054382324, 54.46445441246033, 54.3529052734375, 54.47009086608887, 54.47942352294922, 54.2951385974884, 54.36893892288208, 55.0146324634552, 54.674785137176514, 54.98819875717163]

Experiment 29 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Precision: [0.1967213114754

## jEdit 3.2 - 4.0

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Precision: [0.13533834586466165, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.13533834586466165, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573] 
Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.23841059602649003, 0.23762376237623764, 0.23762376237623764, 0.23762376237623764, 0.23762376237623764, 0.23762376237623764, 0.23762376237623764, 0.23762376237623764, 0.23762376237623764, 0.23841059602649003, 0.23762376237623764, 0.23762376237623764, 0.23762376237623764, 0.23762376237623764, 0.23762

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 5, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 5 : {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.13533834586466165, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573, 0.1348314606741573] 
Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.23762376237623764, 0.23762376237623764, 0.23762376237623764, 0.23762376237623764, 0.23762376237623764, 0.2376237

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.0, 0.17177914110429449, 0.0, 0.0, 0.0, 0.0, 0.15625, 0.0, 0.1657754010695187, 0.0, 0.0, 0.0, 0.15920398009950248, 0.16243654822335024, 0.0, 0.0, 0.2692307692307692, 0.0, 0.15897435897435896, 0.16129032258064516] 
Recall: [0.0, 0.7777777777777778, 0.0, 0.0, 0.0, 0.0, 0.8333333333333334, 0.0, 0.8611111111111112, 0.0, 0.0, 0.0, 0.8888888888888888, 0.8888888888888888, 0.0, 0.0, 0.19444444444444445, 0.0, 0.8611111111111112, 0.8333333333333334] 
F1: [0.0, 0.2814070351758794, 0.0, 0.0, 0.0, 0.0, 0.2631578947368421, 0.0, 0.2780269058295964, 0.0, 0.0, 0.0, 0.27004219409282704, 0.27467811158798283, 0.0, 0.0, 0.22580645161290322, 0.0, 0.2683982683982684, 0.27027027027027023] 
Runtime: [53.05729913711548, 53.410929918289185, 53.29293179512024, 53.25200366973877, 52.59806489944458, 53.095993518829346, 53.42378520965576, 53.770747900009155, 53.81784152984619, 53.860652923583984, 54.01435923576355, 55.42571830749512, 54.953362464904785, 54.974602699279785, 54.93632411956787, 55.25959372

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.1791044776119403, 0.1590909090909091, 0.14583333333333334, 0.0, 0.0, 0.0, 0.0, 0.14720812182741116, 0.13218390804597702, 0.0, 0.0, 0.17159763313609466, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.14285714285714285, 0.15172413793103448, 0.16666666666666666] 
Recall: [0.6666666666666666, 0.5833333333333334, 0.9722222222222222, 0.0, 0.0, 0.0, 0.0, 0.8055555555555556, 0.6388888888888888, 0.0, 0.0, 0.8055555555555556, 0.0, 0.6666666666666666, 0.0, 0.0, 0.0, 0.05555555555555555, 0.6111111111111112, 0.8333333333333334] 
F1: [0.2823529411764706, 0.25, 0.2536231884057971, 0.0, 0.0, 0.0, 0.0, 0.24892703862660945, 0.21904761904761905, 0.0, 0.0, 0.28292682926829266, 0.0, 0.26666666666666666, 0.0, 0.0, 0.0, 0.08, 0.24309392265193372, 0.2777777777777778] 
Runtime: [54.65948271751404, 54.94054365158081, 54.947550773620605, 55.253504037857056, 55.55407476425171, 55.67673373222351, 55.74191617965698, 55.644649267196655, 55.95590114593506, 55.82739448547363, 55.7170934677124, 55.96401858329

## jEdit 4.0-4.1

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704, 0.13703703703703704] 
Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.24104234527687296, 0.24104234527687296, 0.24104234527687296, 0.24104234527687296, 0.241042345

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.0, 0.0, 0.0, 0.16371681415929204, 0.137546468401487, 0.0, 0.0, 0.15163934426229508, 0.0, 0.17989417989417988, 0.14624505928853754, 0.16831683168316833, 0.16517857142857142, 0.1572052401746725, 0.17010309278350516, 0.0, 0.21, 0.14122137404580154, 0.18518518518518517, 0.32142857142857145] 
Recall: [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.918918918918919, 1.0, 0.4594594594594595, 1.0, 0.972972972972973, 0.8918918918918919, 0.0, 0.5675675675675675, 1.0, 0.9459459459459459, 0.24324324324324326] 
F1: [0.0, 0.0, 0.0, 0.2813688212927757, 0.24183006535947715, 0.0, 0.0, 0.2633451957295374, 0.0, 0.30088495575221236, 0.2551724137931034, 0.24637681159420294, 0.2835249042145594, 0.27067669172932335, 0.2857142857142857, 0.0, 0.30656934306569344, 0.24749163879598665, 0.30973451327433627, 0.27692307692307694] 
Runtime: [53.29599905014038, 52.68546772003174, 53.078967332839966, 52.80869388580322, 52.93080282211304, 52.75362467765808, 52.755696296691895, 53.304023027420044, 53.139914

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23076923076923078, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.42857142857142855, 0.0, 0.0, 0.0] 
Recall: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08108108108108109, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08108108108108109, 0.0, 0.0, 0.0] 
F1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13636363636363638, 0.0, 0.0, 0.0] 
Runtime: [47.15962338447571, 47.197951316833496, 47.2150673866272, 47.29680156707764, 47.50034475326538, 47.53598213195801, 47.5048394203186, 47.552921772003174, 47.56815814971924, 47.58799338340759, 47.70128655433655, 47.87916445732117, 47.79951000213623, 47.86759376525879, 47.94406318664551, 48.17122292518616, 48.109110593795776, 48.21579909324646, 48.179306507110596, 48.159586906433105]

Experiment 29 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Precision: [0.24460431654

## log4j 1.0 - 1.1

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.19101123595505617, 0.19767441860465115, 0.19101123595505617, 0.19101123595505617, 0.19101123595505617, 0.19101123595505617, 0.19101123595505617, 0.19101123595505617, 0.07692307692307693, 0.19101123595505617, 0.19444444444444445, 0.2028985507246377, 0.19101123595505617, 0.19101123595505617, 0.19101123595505617, 0.234375, 0.2, 0.19444444444444445, 0.19101123595505617, 0.2] 
Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.058823529411764705, 1.0, 0.8235294117647058, 0.8235294117647058, 1.0, 1.0, 1.0, 0.8823529411764706, 0.8235294117647058, 0.8235294117647058, 1.0, 1.0] 
F1: [0.32075471698113206, 0.3300970873786408, 0.3207

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.1794871794871795, 0.2777777777777778, 0.19101123595505617, 0.22807017543859648, 0.2631578947368421, 0.22727272727272727, 0.22388059701492538, 0.0, 0.19318181818181818, 0.20512820512820512, 0.19101123595505617, 0.038461538461538464, 0.20481927710843373, 0.125, 0.18604651162790697, 0.0, 0.13043478260869565, 0.2727272727272727, 0.0, 0.125] 
Recall: [0.8235294117647058, 0.29411764705882354, 1.0, 0.7647058823529411, 0.5882352941176471, 0.8823529411764706, 0.8823529411764706, 0.0, 1.0, 0.9411764705882353, 1.0, 0.058823529411764705, 1.0, 0.23529411764705882, 0.9411764705882353, 0.0, 0.17647058823529413, 0.7058823529411765, 0.0, 0.23529411764705882] 
F1: [0.2947368421052632, 0.28571428571428575, 0.32075471698113206, 0.3513513513513513, 0.3636363636363636, 0.3614457831325301, 0.35714285714285715, 0.0, 0.3238095238095238, 0.3368421052631579, 0.32075471698113206, 0.04651162790697675, 0.34, 0.16326530612244897, 0.3106796116504854, 0.0, 0.15, 0.39344262295081966, 0.0, 0.16326530612244

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.0, 0.17721518987341772, 0.24, 0.18421052631578946, 0.2916666666666667, 0.0, 0.19444444444444445, 0.0, 0.0, 0.0, 0.4090909090909091, 0.0, 0.2641509433962264, 0.3, 0.0, 0.29545454545454547, 0.0, 0.20588235294117646, 0.0, 0.0] 
Recall: [0.0, 0.8235294117647058, 0.7058823529411765, 0.8235294117647058, 0.8235294117647058, 0.0, 0.8235294117647058, 0.0, 0.0, 0.0, 0.5294117647058824, 0.0, 0.8235294117647058, 0.8823529411764706, 0.0, 0.7647058823529411, 0.0, 0.4117647058823529, 0.0, 0.0] 
F1: [0.0, 0.2916666666666667, 0.3582089552238806, 0.3010752688172043, 0.4307692307692308, 0.0, 0.3146067415730337, 0.0, 0.0, 0.0, 0.46153846153846156, 0.0, 0.4, 0.44776119402985076, 0.0, 0.42622950819672134, 0.0, 0.2745098039215686, 0.0, 0.0] 
Runtime: [52.39200162887573, 52.682931661605835, 52.71328163146973, 52.820712089538574, 52.91065049171448, 53.75531482696533, 53.313533782958984, 53.63591980934143, 53.236125230789185, 53.46481680870056, 53.50255370140076, 53.738985776901245, 53.62643504142

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.07142857142857142, 0.3333333333333333, 0.0, 0.0, 0.25, 0.0, 0.22916666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.42857142857142855, 0.28, 0.0, 0.2608695652173913, 0.0, 0.25] 
Recall: [0.058823529411764705, 0.058823529411764705, 0.0, 0.0, 0.11764705882352941, 0.0, 0.6470588235294118, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17647058823529413, 0.8235294117647058, 0.0, 0.35294117647058826, 0.0, 0.23529411764705882] 
F1: [0.06451612903225808, 0.1, 0.0, 0.0, 0.15999999999999998, 0.0, 0.3384615384615385, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.417910447761194, 0.0, 0.3, 0.0, 0.24242424242424243] 
Runtime: [64.38787031173706, 64.36829686164856, 64.40123200416565, 64.49483060836792, 64.72266149520874, 64.76579761505127, 64.92688155174255, 65.10012626647949, 65.18105125427246, 65.35569906234741, 65.37848377227783, 65.46488881111145, 65.39759302139282, 65.6761212348938, 65.70907211303711, 66.06769466400146, 67.35811948776245, 67.31309604644775, 66.29699563980103, 66.36525869369

## lucene 2.0-2.2

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.37037037037037035, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244, 0.36809815950920244] 
Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.538116591928251, 0.538116591928251, 0.538116591928251, 0.538116591928251, 0.5405405405405406,

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.36607142857142855, 0.38461538461538464, 0.0, 0.5, 0.3900709219858156, 0.36809815950920244, 0.36764705882352944, 0.36619718309859156, 0.366412213740458, 0.38926174496644295, 0.3684210526315789, 0.3710691823899371, 0.37341772151898733, 0.3888888888888889, 0.5, 0.0, 0.2857142857142857, 0.2727272727272727, 0.3710691823899371, 0.38095238095238093] 
Recall: [0.6833333333333333, 0.8333333333333334, 0.0, 0.06666666666666667, 0.9166666666666666, 1.0, 0.8333333333333334, 0.8666666666666667, 0.8, 0.9666666666666667, 0.9333333333333333, 0.9833333333333333, 0.9833333333333333, 0.11666666666666667, 0.016666666666666666, 0.0, 0.06666666666666667, 0.05, 0.9833333333333333, 0.8] 
F1: [0.4767441860465116, 0.5263157894736842, 0.0, 0.11764705882352941, 0.5472636815920398, 0.538116591928251, 0.5102040816326531, 0.514851485148515, 0.5026178010471204, 0.5550239234449761, 0.5283018867924527, 0.5388127853881278, 0.5412844036697249, 0.17948717948717952, 0.03225806451612903, 0.0, 0.1081081081081081

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.375, 0.3504273504273504, 0.5555555555555556, 0.37815126050420167, 0.35185185185185186, 0.0, 0.3333333333333333, 0.3858267716535433, 0.0, 0.35714285714285715, 0.0, 0.34951456310679613, 0.2857142857142857, 0.4423076923076923, 0.36036036036036034, 0.3829787234042553, 0.25, 0.3333333333333333, 0.2608695652173913, 0.37894736842105264] 
Recall: [0.85, 0.6833333333333333, 0.16666666666666666, 0.75, 0.6333333333333333, 0.0, 0.016666666666666666, 0.8166666666666667, 0.0, 0.75, 0.0, 0.6, 0.03333333333333333, 0.38333333333333336, 0.6666666666666666, 0.9, 0.016666666666666666, 0.03333333333333333, 0.1, 0.6] 
F1: [0.520408163265306, 0.46327683615819215, 0.2564102564102564, 0.5027932960893855, 0.4523809523809524, 0.0, 0.031746031746031744, 0.5240641711229946, 0.0, 0.48387096774193544, 0.0, 0.441717791411043, 0.05970149253731343, 0.41071428571428575, 0.4678362573099415, 0.5373134328358209, 0.03125, 0.0606060606060606, 0.14457831325301204, 0.4645161290322581] 
Runtime: [66.30026888847351

## lucene 2.2-2.4

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.3333333333333333, 0.33495145631067963, 0.33170731707317075, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963] 
Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9855072463768116, 1.0, 0.9855072463768116, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.5018181818181818, 0.5018181818181818, 0.5018181818181818, 0.5018

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.33495145631067963, 0.33495145631067963, 0.335, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963, 0.33495145631067963] 
Recall: [1.0, 1.0, 0.9710144927536232, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.5018181818181818, 0.5018181818181818, 0.49814126394052044, 0.5018181818181818, 0.50181818181

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.3316831683168317, 0.3404255319148936, 0.32786885245901637, 0.35911602209944754, 0.32673267326732675, 0.35106382978723405, 0.3388888888888889, 0.36363636363636365, 0.32673267326732675, 0.32989690721649484, 0.3687150837988827, 0.3548387096774194, 0.0, 0.21739130434782608, 0.37209302325581395, 0.3567567567567568, 0.3399014778325123, 0.3384615384615385, 0.3351063829787234, 0.35119047619047616] 
Recall: [0.9710144927536232, 0.927536231884058, 0.8695652173913043, 0.9420289855072463, 0.9565217391304348, 0.9565217391304348, 0.8840579710144928, 0.9855072463768116, 0.9565217391304348, 0.927536231884058, 0.9565217391304348, 0.9565217391304348, 0.0, 0.07246376811594203, 0.927536231884058, 0.9565217391304348, 1.0, 0.9565217391304348, 0.9130434782608695, 0.855072463768116] 
F1: [0.4944649446494465, 0.4980544747081712, 0.4761904761904761, 0.52, 0.48708487084870844, 0.5136186770428015, 0.48995983935742976, 0.53125, 0.48708487084870844, 0.4866920152091254, 0.5322580645161291, 0.5176470588

## poi 1.5 - 2.5

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3584905660377358, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495, 0.3598130841121495] 
Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.987012987012987, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.529209621993127, 0.529209621993127, 0.529209621993127, 0.529209621993127, 0.529209621993127, 0.5292

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.5, 0.39473684210526316, 0.39267015706806285, 0.4378698224852071, 0.42441860465116277, 0.0, 0.41530054644808745, 0.4161849710982659, 0.38860103626943004, 0.4111111111111111, 0.42196531791907516, 0.35714285714285715, 0.43452380952380953, 0.5, 0.36666666666666664, 0.423728813559322, 0.44785276073619634, 0.0, 0.39790575916230364, 0.40641711229946526] 
Recall: [0.025974025974025976, 0.974025974025974, 0.974025974025974, 0.961038961038961, 0.948051948051948, 0.0, 0.987012987012987, 0.935064935064935, 0.974025974025974, 0.961038961038961, 0.948051948051948, 0.974025974025974, 0.948051948051948, 0.012987012987012988, 1.0, 0.974025974025974, 0.948051948051948, 0.0, 0.987012987012987, 0.987012987012987] 
F1: [0.04938271604938272, 0.5617977528089888, 0.5597014925373135, 0.6016260162601625, 0.5863453815261045, 0.0, 0.5846153846153848, 0.576, 0.5555555555555556, 0.5758754863813229, 0.584, 0.5226480836236934, 0.5959183673469388, 0.025316455696202535, 0.5365853658536585, 0.5905511811023

## poi 2.5 - 3.0

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Precision: [0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.6025236593059937, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304] 
Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9502487562189055, 1.0, 1.0, 1.0] 
F1: [0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.7

In [0]:
for i, config in enumerate(get_experimental_config()):
    if i + 1 < 6:
        continue
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 6 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'activation': 'relu'}













Precision: [0.5552486187845304, 0.5552486187845304, 0.5848484848484848, 0.569364161849711, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.610223642172524, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.5552486187845304, 0.6129032258064516, 0.5552486187845304, 0.6111111111111112, 0.5843373493975904] 
Recall: [1.0, 1.0, 0.9601990049751243, 0.9800995024875622, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9502487562189055, 1.0, 1.0, 1.0, 1.0, 1.0, 0.945273631840796, 1.0, 0.9303482587064676, 0.9651741293532339] 
F1: [0.714031971580817, 0.714031971580817, 0.7269303201506592, 0.720292504570384, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.714031971580817, 0.7431906614

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.60625, 0.5808383233532934, 0.6577946768060836, 0.5969696969696969, 1.0, 0.6058631921824105, 0.5945121951219512, 0.6, 0.0, 0.6301369863013698, 0.5968253968253968, 1.0, 0.0, 0.5963302752293578, 0.579250720461095, 0.6233333333333333, 0.6573426573426573, 0.5852941176470589, 0.5969230769230769, 0.6057692307692307] 
Recall: [0.9651741293532339, 0.9651741293532339, 0.8606965174129353, 0.9800995024875622, 0.004975124378109453, 0.9253731343283582, 0.9701492537313433, 0.14925373134328357, 0.0, 0.9154228855721394, 0.9353233830845771, 0.004975124378109453, 0.0, 0.9701492537313433, 1.0, 0.9303482587064676, 0.9353233830845771, 0.9900497512437811, 0.9651741293532339, 0.9402985074626866] 
F1: [0.744721689059501, 0.7252336448598131, 0.7456896551724138, 0.7419962335216573, 0.0099009900990099, 0.7322834645669292, 0.7372400756143667, 0.23904382470119523, 0.0, 0.7464503042596349, 0.7286821705426357, 0.0099009900990099, 0.0, 0.7386363636363635, 0.7335766423357664, 0.7465069860279441, 0.7720739

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.5893719806763285, 0.6065573770491803, 0.2972972972972973, 0.6153846153846154, 1.0, 0.1111111111111111, 0.0, 0.6515151515151515, 0.5, 0.5862068965517241, 0.5352941176470588, 0.6305084745762712, 0.0, 0.6755725190839694, 0.5811209439528023, 0.625, 1.0, 1.0, 0.7037037037037037, 0.5968253968253968] 
Recall: [0.6069651741293532, 0.9203980099502488, 0.05472636815920398, 0.9154228855721394, 0.004975124378109453, 0.009950248756218905, 0.0, 0.8557213930348259, 0.009950248756218905, 0.9303482587064676, 0.4527363184079602, 0.9253731343283582, 0.0, 0.8805970149253731, 0.9800995024875622, 0.024875621890547265, 0.01990049751243781, 0.004975124378109453, 0.7562189054726368, 0.9353233830845771] 
F1: [0.5980392156862744, 0.7312252964426877, 0.09243697478991596, 0.7360000000000001, 0.0099009900990099, 0.018264840182648404, 0.0, 0.7397849462365592, 0.01951219512195122, 0.7192307692307691, 0.4905660377358491, 0.75, 0.0, 0.7645788336933044, 0.7296296296296296, 0.04784688995215311, 0.0390243902

## synapse 1.0-1.1

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.19801980198019803, 0.24, 0.19801980198019803, 0.19801980198019803, 0.19801980198019803, 0.19801980198019803, 0.20100502512562815, 0.19801980198019803, 0.19801980198019803, 0.19801980198019803, 0.2057142857142857, 0.19801980198019803, 0.19387755102040816, 0.18974358974358974, 0.19801980198019803, 0.19801980198019803, 0.19801980198019803, 0.19900497512437812, 0.19801980198019803, 0.19801980198019803] 
Recall: [1.0, 0.45, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 0.95, 0.925, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.3305785123966942, 0.3130434782608696, 0.3305785123966942, 0.3305785123966942, 0.3305785123966942, 0.33057

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.24242424242424243, 0.19801980198019803, 0.19607843137254902, 0.27692307692307694, 0.18421052631578946, 0.21428571428571427, 0.25274725274725274, 0.0, 0.18781725888324874, 0.32, 0.195, 0.25, 0.19900497512437812, 0.35, 0.24107142857142858, 0.19801980198019803, 0.195, 0.20634920634920634, 0.21153846153846154, 0.23636363636363636] 
Recall: [0.2, 1.0, 0.5, 0.45, 0.175, 0.525, 0.575, 0.0, 0.925, 0.4, 0.975, 0.125, 1.0, 0.175, 0.675, 1.0, 0.975, 0.975, 0.55, 0.325] 
F1: [0.21917808219178084, 0.3305785123966942, 0.28169014084507044, 0.34285714285714286, 0.1794871794871795, 0.30434782608695654, 0.3511450381679389, 0.0, 0.31223628691983124, 0.35555555555555557, 0.325, 0.16666666666666666, 0.3319502074688797, 0.23333333333333334, 0.35526315789473684, 0.3305785123966942, 0.325, 0.34061135371179035, 0.3055555555555555, 0.27368421052631575] 
Runtime: [33.125465393066406, 33.00327515602112, 33.18960499763489, 33.32957077026367, 33.4914755821228, 34.11032462120056, 33.16801857948303, 33.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.208, 0.18333333333333332, 0.19801980198019803, 0.21428571428571427, 0.20714285714285716, 0.18309859154929578, 0.16129032258064516, 0.19708029197080293, 0.22340425531914893, 0.20714285714285716, 0.20300751879699247, 0.2158273381294964, 0.21212121212121213, 0.24242424242424243, 0.2727272727272727, 0.0, 0.3125, 0.2, 0.21621621621621623, 0.19424460431654678] 
Recall: [0.65, 0.55, 1.0, 0.9, 0.725, 0.65, 0.125, 0.675, 0.525, 0.725, 0.675, 0.75, 0.525, 0.6, 0.525, 0.0, 0.5, 0.5, 0.4, 0.675] 
F1: [0.3151515151515151, 0.27499999999999997, 0.3305785123966942, 0.3461538461538461, 0.32222222222222224, 0.28571428571428575, 0.14084507042253522, 0.3050847457627119, 0.3134328358208955, 0.32222222222222224, 0.31213872832369943, 0.33519553072625696, 0.302158273381295, 0.3453237410071942, 0.358974358974359, 0.0, 0.38461538461538464, 0.28571428571428575, 0.2807017543859649, 0.30167597765363136] 
Runtime: [35.292993783950806, 34.340728521347046, 35.30482745170593, 35.76355719566345, 34.897671

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [1.0, 0.0, 0.3333333333333333, 1.0, 0.3673469387755102, 0.0, 0.0, 0.16666666666666666, 0.1951219512195122, 1.0, 0.0, 0.25, 0.0, 1.0, 1.0, 0.25, 0.3333333333333333, 0.0, 0.2857142857142857, 0.26666666666666666] 
Recall: [0.025, 0.0, 0.025, 0.025, 0.45, 0.0, 0.0, 0.025, 0.4, 0.025, 0.0, 0.025, 0.0, 0.025, 0.025, 0.025, 0.025, 0.0, 0.05, 0.1] 
F1: [0.04878048780487806, 0.0, 0.046511627906976744, 0.04878048780487806, 0.4044943820224719, 0.0, 0.0, 0.04347826086956522, 0.26229508196721313, 0.04878048780487806, 0.0, 0.045454545454545456, 0.0, 0.04878048780487806, 0.04878048780487806, 0.045454545454545456, 0.046511627906976744, 0.0, 0.0851063829787234, 0.14545454545454545] 
Runtime: [43.1983847618103, 43.596187114715576, 44.74751663208008, 45.62216925621033, 44.90885281562805, 43.90974807739258, 44.34708070755005, 44.08775758743286, 45.064056158065796, 45.20667052268982, 44.53957438468933, 45.04559516906738, 45.28035640716553, 45.16306734085083, 45.24400591850281, 45.378509759902954

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.19205298013245034, 0.1956521739130435, 0.3230769230769231, 0.19801980198019803, 0.19597989949748743, 0.3548387096774194, 0.225, 0.21052631578947367, 0.20526315789473684, 0.0, 0.19801980198019803, 0.20540540540540542, 0.19801980198019803, 0.5, 0.1958762886597938, 0.21052631578947367, 0.2246376811594203, 0.20105820105820105, 0.19696969696969696, 0.2376237623762376] 
Recall: [0.725, 0.9, 0.525, 1.0, 0.975, 0.275, 0.675, 0.7, 0.975, 0.0, 1.0, 0.95, 1.0, 0.2, 0.95, 0.7, 0.775, 0.95, 0.325, 0.6] 
F1: [0.3036649214659686, 0.3214285714285714, 0.4, 0.3305785123966942, 0.3263598326359833, 0.3098591549295775, 0.3375, 0.3236994219653179, 0.3391304347826087, 0.0, 0.3305785123966942, 0.33777777777777784, 0.3305785123966942, 0.28571428571428575, 0.32478632478632474, 0.3236994219653179, 0.34831460674157305, 0.33187772925764186, 0.2452830188679245, 0.3404255319148936] 
Runtime: [46.332292318344116, 45.403544902801514, 45.55325627326965, 45.300963163375854, 47.868948459625244, 46.745948791

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
for i, config in enumerate(get_experimental_config()):
    if i + 1 < 27:
        continue
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 27 : {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [1.0, 0.2857142857142857, 0.5, 0.2839506172839506, 0.18333333333333332, 0.25, 0.2755102040816326, 0.3953488372093023, 0.22641509433962265, 0.5, 0.18032786885245902, 0.0, 0.2916666666666667, 0.22950819672131148, 0.125, 0.0, 0.2987012987012987, 1.0, 1.0, 0.40476190476190477] 
Recall: [0.025, 0.3, 0.025, 0.575, 0.275, 0.4, 0.675, 0.425, 0.3, 0.025, 0.275, 0.0, 0.35, 0.35, 0.025, 0.0, 0.575, 0.025, 0.025, 0.425] 
F1: [0.04878048780487806, 0.2926829268292683, 0.047619047619047616, 0.38016528925619836, 0.21999999999999997, 0.3076923076923077, 0.3913043478260869, 0.40963855421686746, 0.25806451612903225, 0.047619047619047616, 0.21782178217821785, 0.0, 0.31818181818181823, 0.2772277227722772, 0.04166666666666667, 0.0, 0.39316239316239315, 0.04878048780487806, 0.04878048780487806, 0.41463414634146345] 
Runtime: [1.762037754058838, 1.0760724544525146, 1.1607961654663086, 1.175285816192627, 1.2822234630584717, 1.5157732963562012, 1.4398250579833984, 1.4998438358306885, 1.56137871742248

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## synapse 1.1 - 1.2

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Precision: [0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.29508196721311475, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423] 
Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6923076923076923, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.3795620437956204, 0.3795620437956204, 0.3795620437956204, 0.3795620437956204, 0.3795620437956204, 0.3795620437956204, 0.3795620437956204, 0.3795620437956204, 0.3795620437956204, 0.41379

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.24623115577889448, 0.0, 0.27439024390243905, 0.24731182795698925, 0.2604166666666667, 0.2512820512820513, 0.26262626262626265, 0.2603550295857988, 0.2604166666666667, 0.2919254658385093, 0.0, 0.5, 0.0, 0.25925925925925924, 0.2867647058823529, 0.26063829787234044, 0.2777777777777778, 0.0, 0.28859060402684567, 0.0] 
Recall: [0.9423076923076923, 0.0, 0.8653846153846154, 0.8846153846153846, 0.9615384615384616, 0.9423076923076923, 1.0, 0.8461538461538461, 0.9615384615384616, 0.9038461538461539, 0.0, 0.019230769230769232, 0.0, 0.9423076923076923, 0.75, 0.9423076923076923, 0.9615384615384616, 0.0, 0.8269230769230769, 0.0] 
F1: [0.39043824701195223, 0.0, 0.4166666666666667, 0.38655462184873945, 0.4098360655737705, 0.39676113360323884, 0.4160000000000001, 0.39819004524886875, 0.4098360655737705, 0.44131455399061026, 0.0, 0.037037037037037035, 0.0, 0.40663900414937754, 0.41489361702127664, 0.4083333333333333, 0.43103448275862066, 0.0, 0.42786069651741293, 0.0] 
Runtime: [52.4139022

In [0]:
for i, config in enumerate(get_experimental_config()):
    if i + 1 < 24:
        continue
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 24 : {'deep_learner': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.23636363636363636, 0.23423423423423423, 0.23684210526315788, 0.23423423423423423, 0.25595238095238093, 0.27049180327868855, 0.32978723404255317, 0.30833333333333335, 0.2681564245810056, 0.23423423423423423, 0.23423423423423423, 0.25773195876288657, 0.23423423423423423, 0.23195876288659795, 0.23423423423423423, 0.23423423423423423, 0.23423423423423423, 0.24731182795698925, 0.23423423423423423, 0.22282608695652173] 
Recall: [1.0, 1.0, 0.8653846153846154, 1.0, 0.8269230769230769, 0.6346153846153846, 0.5961538461538461, 0.7115384615384616, 0.9230769230769231, 1.0, 1.0, 0.9615384615384616, 1.0, 0.8653846153846154, 1.0, 1.0, 1

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.25, 0.3333333333333333, 0.3177570093457944, 0.0, 0.14285714285714285, 0.25, 0.3333333333333333, 0.0, 0.0, 0.0, 0.4, 0.49122807017543857, 0.3153153153153153, 0.0, 0.0, 0.35294117647058826, 0.0, 0.4, 0.0, 0.2] 
Recall: [0.038461538461538464, 0.038461538461538464, 0.6538461538461539, 0.0, 0.038461538461538464, 0.11538461538461539, 0.019230769230769232, 0.0, 0.0, 0.0, 0.038461538461538464, 0.5384615384615384, 0.6730769230769231, 0.0, 0.0, 0.11538461538461539, 0.0, 0.038461538461538464, 0.0, 0.038461538461538464] 
F1: [0.06666666666666668, 0.06896551724137931, 0.4276729559748428, 0.0, 0.060606060606060615, 0.15789473684210525, 0.03636363636363637, 0.0, 0.0, 0.0, 0.07017543859649124, 0.5137614678899082, 0.4294478527607362, 0.0, 0.0, 0.1739130434782609, 0.0, 0.07017543859649124, 0.0, 0.06451612903225806] 
Runtime: [8.818812847137451, 8.938392877578735, 9.103600978851318, 9.09008502960205, 9.202089071273804, 9.337066888809204, 9.435756206512451, 9.480075359344482, 9.5956614017486

## xalan 2.4 - 2.5

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.47754137115839246, 0.4165497896213184, 0.4165497896213184, 0.4165497896213184, 0.4165497896213184, 0.4165497896213184, 0.4165497896213184, 0.4165497896213184, 0.4165497896213184, 0.4165497896213184, 0.4914772727272727, 0.4165497896213184, 0.4165497896213184, 0.4876237623762376, 0.4165497896213184, 0.4165497896213184, 0.4165497896213184, 0.4165497896213184, 0.4165497896213184, 0.4165497896213184] 
Recall: [0.6801346801346801, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5824915824915825, 1.0, 1.0, 0.6632996632996633, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 
F1: [0.5611111111111111, 0.5881188118811881, 0.5881188118811881, 0.5881188

In [0]:
for i, config in enumerate(get_experimental_config()):
    if i + 1 < 5:
        continue
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 5 : {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.4975, 0.4165497896213184, 0.4165497896213184, 0.5, 0.4165497896213184, 0.4165497896213184, 0.4165497896213184, 0.4939759036144578, 0.4165497896213184, 0.4157303370786517, 0.4165497896213184, 0.4165497896213184, 0.47544642857142855, 0.4165497896213184, 0.4845605700712589, 0.5111821086261981, 0.4165497896213184, 0.49829351535836175, 0.4165497896213184, 0.5142857142857142] 
Recall: [0.67003367003367, 1.0, 1.0, 0.5521885521885522, 1.0, 1.0, 1.0, 0.5521885521885522, 1.0, 0.9966329966329966, 1.0, 1.0, 0.7171717171717171, 1.0, 0.6868686868686869, 0.5387205387205387, 1.0, 0.49158249158249157, 1.0, 0.48484848484848486] 
F1: [0.571

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.0, 0.45714285714285713, 0.46124031007751937, 0.46464646464646464, 0.0, 0.20833333333333334, 0.4375, 0.4621212121212121, 0.0, 0.4262295081967213, 0.0, 0.4154929577464789, 0.1111111111111111, 0.0, 0.4371482176360225, 0.0, 0.0, 0.46616541353383456, 0.477326968973747, 0.0] 
Recall: [0.0, 0.26936026936026936, 0.4006734006734007, 0.7744107744107744, 0.0, 0.016835016835016835, 0.11784511784511785, 0.6161616161616161, 0.0, 0.8754208754208754, 0.0, 0.19865319865319866, 0.003367003367003367, 0.0, 0.7845117845117845, 0.0, 0.0, 0.6262626262626263, 0.6734006734006734, 0.0] 
F1: [0.0, 0.3389830508474576, 0.4288288288288288, 0.5808080808080808, 0.0, 0.03115264797507788, 0.1856763925729443, 0.5281385281385281, 0.0, 0.5733186328555677, 0.0, 0.2687927107061504, 0.006535947712418301, 0.0, 0.5614457831325301, 0.0, 0.0, 0.5344827586206896, 0.558659217877095, 0.0] 
Runtime: [46.2807981967926, 46.348809003829956, 46.59021353721619, 47.75493335723877, 46.36119222640991, 46.214770793914795, 47.80

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.0, 0.5964912280701754, 1.0, 0.35714285714285715, 0.75, 0.0, 0.0, 0.375, 0.0, 0.0, 0.5, 0.0, 0.0, 0.75, 0.0, 0.3076923076923077, 0.0, 0.0, 0.0, 0.45] 
Recall: [0.0, 0.11447811447811448, 0.003367003367003367, 0.016835016835016835, 0.010101010101010102, 0.0, 0.0, 0.010101010101010102, 0.0, 0.0, 0.003367003367003367, 0.0, 0.0, 0.020202020202020204, 0.0, 0.013468013468013467, 0.0, 0.0, 0.0, 0.030303030303030304] 
F1: [0.0, 0.192090395480226, 0.006711409395973153, 0.03215434083601287, 0.01993355481727575, 0.0, 0.0, 0.019672131147540985, 0.0, 0.0, 0.006688963210702341, 0.0, 0.0, 0.03934426229508197, 0.0, 0.025806451612903226, 0.0, 0.0, 0.0, 0.056782334384858045] 
Runtime: [47.991556882858276, 47.98408341407776, 49.02143836021423, 48.34882974624634, 48.02203011512756, 48.51090049743652, 48.675321102142334, 48.56051683425903, 50.92435956001282, 49.302762031555176, 49.06991982460022, 48.78130602836609, 49.85871624946594, 50.125595569610596, 50.39293622970581, 51.55729627609253, 50.

In [0]:
for i, config in enumerate(get_experimental_config()):
    if i + 1 < 29:
        continue
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 29 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








Precision: [0.5276381909547738, 0.5135135135135135, 0.5185185185185185, 0.49166666666666664, 0.5, 0.5061728395061729, 0.5826086956521739, 0.49122807017543857, 0.5131578947368421, 0.5341614906832298, 0.5302013422818792, 0.51875, 0.5347222222222222, 0.5163398692810458, 0.5925925925925926, 0.53, 0.5401459854014599, 0.5342465753424658, 0.5233644859813084, 0.5073891625615764] 
Recall: [0.35353535353535354, 0.3838383838383838, 0.2828282828282828, 0.39730639730639733, 0.3501683501683502, 0.2760942760942761, 0.2255892255892256, 0.2828282828282828, 0.3939393939393939, 0.2895622895622896, 0.265993265993266, 0.27946127946127947, 0.25925925925925924, 0.265993265993266, 0.26936026936026936, 0.3569023569023569, 0.2491582491582

## xerces 1.2 - 1.3

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Precision: [0.06569343065693431, 0.06569343065693431, 0.06569343065693431, 0.11206896551724138, 0.06569343065693431, 0.06569343065693431, 0.06569343065693431, 0.08727272727272728, 0.06569343065693431, 0.06569343065693431, 0.08518518518518518, 0.06388206388206388, 0.08414239482200647, 0.09811320754716982, 0.06569343065693431, 0.06569343065693431, 0.06569343065693431, 0.06569343065693431, 0.06569343065693431, 0.059278350515463915] 
Recall: [1.0, 1.0, 1.0, 0.9629629629629629, 1.0, 1.0, 1.0, 0.8888888888888888, 1.0, 1.0, 0.8518518518518519, 0.9629629629629629, 0.9629629629629629, 0.9629629629629629, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8518518518518519] 
F1: [0.12328767123287672, 0.12328767123287672, 0.12328767123287672, 0.20077220077220076, 0.1232876712

In [0]:
for i, config in enumerate(get_experimental_config()):
    if i + 1 < 13:
        continue
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 13 : {'deep_learner': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'activation': 'relu'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













Precision: [0.08150470219435736, 0.06569343065693431, 0.05507246376811594, 0.10080645161290322, 0.06476683937823834, 0.05958549222797927, 0.06569343065693431, 0.07746478873239436, 0.06569343065693431, 0.08992805755395683, 0.0859106529209622, 0.06527415143603134, 0.06569343065693431, 0.06349206349206349, 0.07961783439490445, 0.06440677966101695, 0.07714285714285714, 0.07911392405063292, 0.0784313725490196, 0.08253968253968254] 
Recall: [0.9629629629629629, 1.0, 0.7037037037037037, 0.9259259259259259, 0.9259259259259259, 0.8518518518518519, 1.0, 0.8148148148148148, 1.0, 0.9259259259259259, 0.9259259259259259, 0.9259259259259259, 1.0, 0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.1511627906976744, 0.05982905982905983, 0.06938775510204082, 0.05913978494623656, 0.0, 0.08540925266903915, 0.12, 0.0, 0.06569343065693431, 0.07204610951008646, 0.05813953488372093, 0.06319702602230483, 0.06111111111111111, 0.04436860068259386, 0.07142857142857142, 0.05945945945945946, 0.06824146981627296, 0.06569343065693431, 0.05234159779614325, 0.02027027027027027] 
Recall: [0.48148148148148145, 0.25925925925925924, 0.6296296296296297, 0.8148148148148148, 0.0, 0.8888888888888888, 0.2222222222222222, 0.0, 1.0, 0.9259259259259259, 0.18518518518518517, 0.6296296296296297, 0.8148148148148148, 0.48148148148148145, 0.07407407407407407, 0.8148148148148148, 0.9629629629629629, 1.0, 0.7037037037037037, 0.1111111111111111] 
F1: [0.23008849557522124, 0.09722222222222222, 0.12499999999999999, 0.11027568922305764, 0.0, 0.15584415584415584, 0.15584415584415584, 0.0, 0.12328767123287672, 0.13368983957219252, 0.08849557522123894, 0.11486486486486486, 0.11369509043927647, 0.081249999999

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.1497005988023952, 0.078125, 0.08571428571428572, 0.0, 0.11949685534591195, 0.1301775147928994, 0.03333333333333333, 0.11797752808988764, 0.0, 0.1513157894736842, 0.12987012987012986, 0.0812720848056537, 0.1270718232044199, 0.0, 0.0, 0.1342281879194631, 0.0948905109489051, 0.1111111111111111, 0.0, 0.12804878048780488] 
Recall: [0.9259259259259259, 0.9259259259259259, 0.5555555555555556, 0.0, 0.7037037037037037, 0.8148148148148148, 0.07407407407407407, 0.7777777777777778, 0.0, 0.8518518518518519, 0.7407407407407407, 0.8518518518518519, 0.8518518518518519, 0.0, 0.0, 0.7407407407407407, 0.9629629629629629, 0.5555555555555556, 0.0, 0.7777777777777778] 
F1: [0.25773195876288657, 0.14409221902017288, 0.1485148514851485, 0.0, 0.20430107526881722, 0.22448979591836735, 0.04597701149425288, 0.20487804878048782, 0.0, 0.25698324022346364, 0.2209944751381215, 0.14838709677419354, 0.22115384615384617, 0.0, 0.0, 0.22727272727272727, 0.17275747508305647, 0.18518518518518515, 0.0, 0.219895

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0] 
Recall: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.037037037037037035, 0.0, 0.0, 0.0, 0.18518518518518517, 0.0, 0.0, 0.0, 0.0] 
F1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06060606060606061, 0.0, 0.0, 0.0, 0.23809523809523808, 0.0, 0.0, 0.0, 0.0] 
Runtime: [26.77480435371399, 26.8545823097229, 27.01724100112915, 26.91836953163147, 27.230698347091675, 27.128894090652466, 27.455604791641235, 27.192625761032104, 27.151041746139526, 27.340696096420288, 27.068888664245605, 27.326155185699463, 27.4242160320282, 27.404660940170288, 27.955477476119995, 27.74852228164673, 27.302682876586914, 27.69454526901245, 27.668386936187744, 27.52167296409607]

Experiment 29 : {'deep_learner': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}
Pre